# Helper Functions Volume 2A: Date Time Processing

Author: Koh Kok Bin  
Date: 10 Dec 2021

This notebook provides aan overall guide to users who have to deal with date/time data. Having a good knowledge of date/time manipulation will greatly reduce the time taken to code the scripts, especially on data cleaning.  

I will try my best to make this as easy to understand for Python users of all levels. Most if not all of the codes used here can be cross-applied to other areas of your work.

Have fun copying!


### <a id = "VOL2A_toc"> Table of contents:  </a>

- [Reference Materials -- This needs to be run so that Parts 1 to n can work.](#VOL2A_refmat)
- [Help on codes](#VOL2A_help)
- [Part 1: Selecting and Indexing data](#VOL2A_selectdata)
- [Data Processing](#VOL2A_dataclean)  
- [Exploratory Data Analysis](#VOL2A_EDA)

Each dataset has its own date/time range. Yours may be in days (2020-01-01), months (2020-01), or years (2020). Some may be hours (2020-01-01 00H), minutes (2020-01-01 00-00), seconds (2020-01-01 000000).  

Note the different symbols used (-, H). Because there's likely a human behind the process to annotate the dates (can be data entry for physical forms, officer who drew up google forms or form.sg), the date format maybe different. Python doesn't know that. Hence, we need to tell it where the separate components are.  

Pandas can usually infer the date from the format that was passed in. We can also specify our own formats. Formats can be found in this internet link:

- https://strftime.org  

Some common formats are listed here. Unlikely you will use others, unless the date and time data is quite messy.

%d (0-padded day of month), %m (0-padded month), %Y (Year), %H (24 Hour clock, 0-padded), %M (0-padded minutes), %S (0-padded seconds).  

Most date time functions follow or make use of native Python package "datetime" to parse dates and times. The most popular function here is:  

- datetime.datetime.strftime  
- datetime.datetime.strptime  

The first datetime is the entire package to manipulate date and time. The second datetime is an object meant to contain all information from date and time object. That object has methods (aka functions) which you can use, including strftime and strptime.  

strftime (in my own words): string format time  
strptime (in my own words): string parse time  

When you want to output time formats, use strftime. If you want to read in time formats, use strptime.